In [1]:
import os
from typing import TypedDict, List, Dict, Annotated
import operator

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

from langgraph.graph import StateGraph, END

gemini_api_key = os.getenv("GEMINI_API_KEY")

The goal of  our project was if we were able to build an agentic system that could produce books, upload them some marketplace ( for us this was Amazon) and if we could somewhat automate the process of writing book. I personally have private interest in literature so I really wonder if this is sensible and if it truly makes sense to automate this process. 

The idea of the workflow is simple.  
The user query is taken. 
Then an outline is generated. 
This outline is then used in a loop to generate the individual chapters.
Then the whole book is concatenated and the book automatically uploaded

In [2]:
# --- 1. Define the State ---
# This state is simplified to focus on the narrative structure: premise -> outline -> chapters.

class BookWriterState(TypedDict):
    """
    Represents the state of our book writing graph.

    Attributes:
        book_topic: The initial high-level topic for the book.
        book_premise: A more detailed premise or summary generated by the LLM.
        book_outline: A dictionary containing the title and a list of chapter titles.
        chapter_index: An index to keep track of the current chapter being written.
        draft_chapters: A list to store the written content of each chapter.
        error: A field to store any errors that occur.
    """
    book_topic: str
    book_premise: str
    book_outline: Dict
    chapter_index: int
    book_title: str
    book_description: str
    # We use Annotated and operator.add to accumulate chapters in the list.
    draft_chapters: Annotated[List[str], operator.add]
    error: str
    end_writing: bool
    author_name: str





As typical in LangGraph you need to define the individual nodes. We have a node for each task of the writing process. We quickly test the LLM here , just so we know it works

In [3]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.2, max_output_tokens=10240,  api_key=gemini_api_key)

##test LLM

response = llm.invoke("Just say 'Hello, World!'")
print(response)
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

    


def generate_premise_node(state: BookWriterState) -> Dict:
    """Generates a compelling book premise from a high-level topic."""
    print("--- Generating Book Premise ---")
    try:
        prompt = PromptTemplate(
            template="Generate a compelling one-paragraph book premise based on this topic: {topic}",
            input_variables=["topic"],
        )
        chain = prompt | llm | str_parser
        print("current book topic:", state["book_topic"])
        premise = chain.invoke({"topic": state["book_topic"]})
        print(f"Generated Premise: {premise}")
        prompt = PromptTemplate(
            template="Generate a nice title for a book based on this premise: {premise}. Just give me the title, nothing else.",
            input_variables=["premise"],
        )
        title_chain = prompt | llm | str_parser
        book_title = title_chain.invoke({"premise": premise})
        print(f"Generated Title: {book_title}")
        
        return {"book_premise": premise, "book_title": book_title} 
    except Exception as e:
        return {"error": f"Failed to generate premise: {e}"}

def generate_outline_node(state: BookWriterState) -> Dict:
    """Generates a book outline based on the premise."""
    print("\n--- Generating Book Outline ---")
    try:
        print(f"now creating outline for premise: {state['book_premise']}")
        prompt = PromptTemplate(
            template="Based on this premise, create a book outline with a title and a list of chapter titles (JSON format) . Premise: {premise}. Use just two chapters for simplicity.", #and a brief description for each chapter
            input_variables=["premise"],
        )
        chain = prompt | llm  | str_parser | json_parser
        outline = chain.invoke({"premise": state["book_premise"]})
        print(f"Generated Outline: {outline}")
        return {"book_outline": outline}
    except Exception as e:
        return {"error": f"Failed to generate outline: {e}"}

def write_chapter_node(state: BookWriterState) -> Dict:
    """Writes the content for a single chapter based on the outline."""
    print("\n--- Writing Chapter ---")
    #print(f"Current State: {state}")
    try:
        outline = state["book_outline"]
        chapter_index = state["chapter_index"]
        chapter_title = outline["chapters"][chapter_index]
        
        prompt = PromptTemplate(
            template="Write the content for Chapter {chapter_num}: '{chapter_title}'. The overall book premise is: {premise}",
            input_variables=["chapter_num", "chapter_title", "premise"],
        )
        chain = prompt | llm | str_parser
        chapter_content = chain.invoke({
            "chapter_num": chapter_index + 1,
            "chapter_title": chapter_title,
            "premise": state["book_premise"]
        })
        print(f"Wrote Content for: {chapter_title}")
        return {"draft_chapters": [chapter_content], "chapter_index": chapter_index + 1}
    except Exception as e:
        return {"error": f"Failed to write chapter: {e}"}


def end_writing_node(state: BookWriterState) -> Dict:
    """Finalizes the writing process."""
    print("\n--- Finalizing Writing Process ---")
    try:
        if state["draft_chapters"]:
            print("Writing completed successfully.")
            return {"draft_chapters": state["draft_chapters"], "end_writing": True}
        else:
            return {"error": "No chapters were written.", "end_writing": True}
    except Exception as e:
        return {"error": f"Failed to finalize writing: {e}", "end_writing": True}
    

"""def write_description_node(state: BookWriterState) -> Dict:
    
    print("\n--- Writing Book Description ---")
    try:
        prompt = PromptTemplate(
            template="This is a book I wrote. You will now receive a json of the individual chapters in order:  {chapters}. It has the title {title}. Please write a compelling description for it. You can use HTML tags to format the description.",
            input_variables=["chapters", "title"],
        )
        chain = prompt | llm | str_parser
        #print(f"Current book outline: {state['book_outline']}")
        description = chain.invoke({"chapters": state["book_outline"], "title": state["book_title"]})
        print(f"Generated Description: {description}")
        return {"book_description": description}
    except Exception as e:
        return {"error": f"Failed to write description: {e}"}"""

def write_description_node(state: BookWriterState) -> Dict:
    print("\n--- Writing Book Description ---")
    try:
        prompt = PromptTemplate(
            template="This is a book I wrote: {draft_chapters} It has the title {title}. Please write a compelling description for it. You can use HTML tags to format the description.  Just give me the description, nothing else.",
            input_variables=["title", "draft_chapters"],  # Include draft_chapters if needed for context
        )
        #print("Starting to format prompt")
        formatted_prompt = prompt.format(title=state.get("book_title", "[Untitled Book]"), draft_chapters=state.get("draft_chapters", []))

        #print(f"Formatted prompt: {formatted_prompt}")

        response = llm.invoke(formatted_prompt)
        #print(f"Raw LLM output: {response}")

        description = str_parser.invoke(response)
        #print(f"Generated Description: {description}")
        return {"book_description": description}
    except Exception as e:
        return {"error": f"Failed to write description: {e}"}





content='Hello, World!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--efd03dfd-b11c-4175-93f1-56984718ecea-0' usage_metadata={'input_tokens': 8, 'output_tokens': 20, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 16}}


In [4]:


def should_continue_writing(state: BookWriterState) -> str:
    """Checks if there are more chapters to write."""
    print("--- Checking for More Chapters ---")
    next_index = state["chapter_index"]
    
    if next_index < len(state["book_outline"]["chapters"]):
        print("Decision: Continue with next chapter.")
        # Update the index in the state to process the next chapter
        return "continue_writing"
    else:
        print("Decision: All chapters written. End process.")
        return "end_writing"



    


In [5]:
# --- 5. Assemble the Graph ---

workflow = StateGraph(BookWriterState)

# Add nodes
workflow.add_node("generate_premise", generate_premise_node)
workflow.add_node("generate_outline", generate_outline_node)
workflow.add_node("write_chapter", write_chapter_node)
workflow.add_node("end_writing", end_writing_node)
workflow.add_node("write_description", write_description_node)

# Define the flow
workflow.set_entry_point("generate_premise")
workflow.add_edge("generate_premise", "generate_outline")
workflow.add_edge("generate_outline", "write_chapter")


# Add the conditional edge for the chapter writing loop
workflow.add_conditional_edges(
    "write_chapter",
    should_continue_writing,# A checkpointer is needed for state updates in conditional edges, but we can manage manually for this simple case
    {
        "continue_writing": "write_chapter", # If 'continue', loop back
        "end_writing": "write_description"               # If 'done', finish
    }
)

workflow.add_edge("write_description", "end_writing")
#workflow.add_edge("write_description", END)  # End the workflow after writing the description

# Compile the graph
app = workflow.compile(checkpointer=None) # A checkpointer is needed for state updates in conditional edges, but we can manage manually for this simple case.



In [6]:
# --- 6. Run the Graph ---


def run_graph(initial_input):
    # Make a mutable copy of the state to track it through the loop
    state = initial_input.copy()
    
    while True:
        # Call the graph with the current state
        output = app.invoke(state)
    
        # If the graph is finished, return the final state
        if output.get("end_writing"):
            print("Graph finished.")
            return output
      
        
    



initial_input = {
    "book_topic": "A space opera about a lost human colony that has evolved.",
    "chapter_index": 0,
    "draft_chapters": [],
    "author_name": "Johannes Barthold",
}

print("--- Starting Book Writer Graph ---")
final_state = run_graph(initial_input)

print("\n--- Graph Finished ---")
print(f"\nTitle: {final_state.get('book_outline', {}).get('title')}")
print("\nPremise:")
print(final_state.get('book_premise'))
print("\nDraft Chapters:")
for i, chapter in enumerate(final_state.get('draft_chapters', [])):
    print(f"--- Chapter {i+1} ---\n{chapter}\n")
print("\nBook Description:")
print(final_state.get('book_description', 'No description generated.'))
print("\nAuthor Name:")
print(final_state.get('author_name', 'No author name provided.'))
print("\n--- Book Writing Process Completed ---")

--- Starting Book Writer Graph ---
--- Generating Book Premise ---
current book topic: A space opera about a lost human colony that has evolved.
Generated Premise: For millennia, the lost colony of Xylos was a myth, a whisper in the void, until a desperate Terran scout ship stumbled upon their verdant, living world. There, humanity had not merely survived, but *evolved*, their descendants – the Sylvans – now a people of living light and interwoven consciousness, their bodies adapted to the planet's unique energies, their minds capable of unparalleled sensory and psionic feats, utterly alien yet undeniably human. But their peaceful, symbiotic existence is shattered by the encroaching cosmic blight, a force consuming star systems and threatening all life; now, the fractured galactic core, desperate and out of options, must convince these transcendent beings – who have long forgotten the stars – to leave their Eden and become the universe's last, improbable hope, forcing them to choose be

Now we have generated our book. We want to format is nicely though so we need to create an EBUP to publish this book.

In [7]:
from ebooklib import epub
from PIL import Image

img = Image.new("RGB", (600, 800), color="skyblue")
img.save("cover.jpg", "JPEG")

def create_epub(state: BookWriterState, filename: str = "book.epub"):
    """Creates an EPUB file from the book state."""
    book = epub.EpubBook()

    # Set metadata
    book.set_title(state.get("book_title", "Untitled Book"))
    book.set_language("en")
    book.add_author(state.get("author_name", "Unknown Author"))

    # Add chapters
    chapter_items = []
    for i, chapter in enumerate(state.get("draft_chapters", [])):
        c = epub.EpubHtml(title=f"Chapter {i+1}", file_name=f"chapter_{i+1}.xhtml", lang="en")
        c.content = f"<h1>Chapter {i+1}</h1><p>{chapter}</p>"
        book.add_item(c)
        chapter_items.append(c)

    # Add navigation
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # Set spine
    book.spine = ['nav'] + chapter_items

    # Add cover image
    try:
        with open("cover.jpg", "rb") as img_file:
            cover_data = img_file.read()
        book.set_cover("cover.jpg", cover_data)
    except FileNotFoundError:
        print("⚠️ Cover image 'cover.jpg' not found. Skipping cover.")

    # Add chapters to book
    for item in chapter_items:
        book.add_item(item)

    # Save the EPUB file
    epub.write_epub(filename, book)
    print(f"✅ EPUB file '{filename}' created successfully.")

create_epub(final_state, "romantic_quest.epub")

ParserError: Document is empty

After creating our EPUB we need to submit this book to Amazon. We can do this in an automatic fashion using Selenium.